In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# layers
import numpy as np

class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx


class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx


class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)
        return dx


class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실함수
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 인코딩 형태)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩 형태일 때
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx

'\nclass Dropout:\n    """\n    http://arxiv.org/abs/1207.0580\n    """\n    def __init__(self, dropout_ratio=0.5):\n        self.dropout_ratio = dropout_ratio\n        self.mask = None\n\n    def forward(self, x, train_flg=True):\n        if train_flg:\n            self.mask = np.random.rand(*x.shape) > self.dropout_ratio\n            return x * self.mask\n        else:\n            return x * (1.0 - self.dropout_ratio)\n\n    def backward(self, dout):\n        return dout * self.mask\n\n\nclass BatchNormalization:\n    """\n    http://arxiv.org/abs/1502.03167\n    """\n    def __init__(self, gamma, beta, momentum=0.9, running_mean=None, running_var=None):\n        self.gamma = gamma\n        self.beta = beta\n        self.momentum = momentum\n        self.input_shape = None # 합성곱 계층은 4차원, 완전연결 계층은 2차원  \n\n        # 시험할 때 사용할 평균과 분산\n        self.running_mean = running_mean\n        self.running_var = running_var  \n        \n        # backward 시에 사용할 중간 데이터\n        self.batch_size 

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))    


def sigmoid_grad(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)
    

def relu(x):
    return np.maximum(0, x)


def relu_grad(x):
    grad = np.zeros(x)
    grad[x>=0] = 1
    return grad
    

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))


def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)


def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size


def softmax_loss(X, t):
    y = softmax(X)
    return cross_entropy_error(y, t)


In [ ]:
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        it.iternext()   
        
    return grad



In [ ]:
# two_layer_net
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
#from common.layers import *
#from common.gradient import numerical_gradient
from collections import OrderedDict


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()           # 순전파, 역전파 사용시 계산 순서 용이하게 하기 위해 딕셔너리에 추가한 순서를 기억하는 OrderedDict()함수 사용
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads



In [ ]:
import random
from glob import glob
import sys, os
sys.path.append(os.pardir)
from matplotlib import image
from PIL import Image
#from dataset.mnist import load_mnist
#from two_layer_net import TwoLayerNet

def change_one_hot_label(X):  # X type : numpy list
    T = np.zeros((X.size, 3))   ## 3으로 수정 후 데이터 원 핫 라벨 수정
    for idx, row in enumerate(T):
        row[X[idx]] = 1
      
    return T


def img_to_np(fpaths,label,m,n,resize=True):  
    img_array = []
    lab_array = []
    for fname in fpaths:
      try:
        # print(fname)
        img = Image.open(fname).convert('RGB')
        if(resize): img = img.resize((m, n))
        img_array.append(np.asarray(img))
        lab_array.append(label)
      except:
        continue
    images = np.array(img_array)
    labels = change_one_hot_label(np.array(lab_array)) #np.array(lab_array)
    return images, labels




In [ ]:
##
volume0_list = glob('/content/drive/My Drive/jeju_volume_data/volume0/*.jpg')
volume1_list = glob('/content/drive/My Drive/jeju_volume_data/volume1/*.jpg')
volume2_list = glob('/content/drive/My Drive/jeju_volume_data/volume2/*.jpg')

index_0 = []
index_1 = []
index_2 = []
balance = min(len(volume0_list), len(volume1_list), len(volume2_list))

for i in range((balance)):
  # if i not in index_0
  index_0.append(volume0_list[random.randint(0,len(volume0_list)-1)])
  index_1.append(volume1_list[random.randint(0,len(volume1_list)-1)])
  index_2.append(volume2_list[random.randint(0,len(volume2_list)-1)])
print(len(index_0))


img_volume0,lab_volume0 = img_to_np(index_0,0,100,100)
img_volume0 = img_volume0.astype(np.float32)/ 255.

img_volume1,lab_volume1 = img_to_np(index_1,1,100,100)
img_volume1 = img_volume1.astype(np.float32)/ 255.

img_volume2,lab_volume2 = img_to_np(index_2,2,100,100)
img_volume2 = img_volume2.astype(np.float32)/ 255.

print(img_volume0.shape)
print(img_volume0.size)

from sklearn.utils import shuffle # 데이터 편향 방지
x_train = np.concatenate((img_volume0, img_volume1, img_volume2), axis=0)
t_train = np.concatenate((lab_volume0, lab_volume1, lab_volume2), axis=0)
x_train, t_train = shuffle(x_train, t_train)
print(x_train.shape)


# test data set
t_volume0_list = glob('/content/drive/My Drive/jeju_volume_data/test1/volume0/*.jpg')
t_volume1_list = glob('/content/drive/My Drive/jeju_volume_data/test1/volume1/*.jpg')
t_volume2_list = glob('/content/drive/My Drive/jeju_volume_data/test1/volume2/*.jpg')

t_index_0 = []
t_index_1 = []
t_index_2 = []
t_balance = min(len(t_volume0_list), len(t_volume1_list), len(t_volume2_list))

for i in range((t_balance)):
  # if i not in index_0
  t_index_0.append(volume0_list[random.randint(0,len(t_volume0_list)-1)])
  t_index_1.append(volume1_list[random.randint(0,len(t_volume1_list)-1)])
  t_index_2.append(volume2_list[random.randint(0,len(t_volume2_list)-1)])
print(len(t_index_0))


t_img_volume0,t_lab_volume0 = img_to_np(t_index_0,0,100,100)
t_img_volume0 = t_img_volume0.astype(np.float32)/ 255.

t_img_volume1,t_lab_volume1 = img_to_np(t_index_1,1,100,100)
t_img_volume1 = t_img_volume1.astype(np.float32)/ 255.

t_img_volume2,t_lab_volume2 = img_to_np(t_index_2,2,100,100)
t_img_volume2 = t_img_volume2.astype(np.float32)/ 255.

print(t_img_volume0.shape)
print(t_img_volume0.size)

from sklearn.utils import shuffle # 데이터 편향 방지
x_test = np.concatenate((t_img_volume0, t_img_volume1, t_img_volume2), axis=0)
t_test = np.concatenate((t_lab_volume0, t_lab_volume1, t_lab_volume2), axis=0)
x_test, t_test = shuffle(x_test, t_test)
print(x_test.shape)



## 테스트 셋 데이터 전처리, 모든 이미지 플랫툰처리 !

200
(200, 100, 100, 3)
6000000
(600, 100, 100, 3)
100
(100, 100, 100, 3)
3000000
(300, 100, 100, 3)


In [ ]:
x_train = np.reshape(x_train,(600,3,100,100))
x_test = np.reshape(x_test,(300,3,100,100))
print(x_train.shape)
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    '''
    change images to 2d mat

    Parameters
    ----------
    input_data: 4d images like mat(number of images, number of channel, height, width)
    filter_h: filter height
    filter_w: filter width
    stride: stride
    pad: padding

    Returns
    -------
    col: 2d mat
    '''

    N, C, H, W = input_data.shape

    out_h = (H + (2 * pad) - filter_h) // stride + 1
    out_w = (W + (2 * pad) - filter_w) // stride + 1
    
    img = np.pad(input_data, [(0, 0), (0, 0),
                              (pad, pad), (pad, pad)], 'constant')
    print(N,C,filter_h,filter_w,out_h,out_w)
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))
    for y in range(filter_h):
        y_max = y + stride * out_h
        for x in range(filter_w):
            x_max = x + stride * out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N * out_h * out_w, -1)

    return col
x_train = im2col(x_train,100,100)
x_test = im2col(x_test,100,100)



(600, 3, 100, 100)
600 3 100 100 1 1
300 3 100 100 1 1


In [ ]:
import matplotlib.pyplot as plt
# 데이터 읽기
#(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=30000, hidden_size=100, output_size=3)

iters_num = 100
train_size = x_train.shape[0]
batch_size = 60
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)
# iter_per_epoch = max(train_size / batch_size, 1)

print(train_size)
print(batch_size)

#x_batch = x_train
#t_batch = t_train

'''
데이터 셔플, 메타 -> 한 리스트에 세가지 타입 넣기 (사진짜로 라벨 따로) -> 그리고 셔플


'''
for i in range(iters_num):
#    batch_mask = np.random.choice(train_size, batch_size)
#    x_batch = x_train[batch_mask]
#    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_train, t_train) # 오차 역전파법(배치x)
    #grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_train, t_train)
    #loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("epoch : ", i)
        print(train_acc, test_acc)


600
60
epoch :  1
0.6516666666666666 0.65
epoch :  11
0.6116666666666667 0.6666666666666666
epoch :  21
0.3333333333333333 0.3333333333333333
epoch :  31
0.3333333333333333 0.3333333333333333
epoch :  41
0.3333333333333333 0.3333333333333333
epoch :  51
0.3333333333333333 0.3333333333333333
epoch :  61
0.34833333333333333 0.35
epoch :  71
0.34833333333333333 0.35
epoch :  81
0.3333333333333333 0.3333333333333333
epoch :  91
0.34833333333333333 0.35
